Pyspark Basics: 
1 . Setting up in colab

In [ ]:
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
#  Download pyspark
#  Get latest version from https://spark.apache.org/downloads.html

!wget  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz

--2022-02-23 09:13:16--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272637746 (260M) [application/x-gzip]
Saving to: ‘spark-3.2.1-bin-hadoop2.7.tgz’

spark-3.2.1-bin-had 100%[===================>] 260.01M   186MB/s    in 1.4s    

2022-02-23 09:13:37 (186 MB/s) - ‘spark-3.2.1-bin-hadoop2.7.tgz’ saved [272637746/272637746]



In [ ]:
!tar xf spark-3.2.1-bin-hadoop2.7.tgz


In [ ]:
# Installing pyspark 

!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
#  Upload stocks.zip to colab directory

In [1]:
%%bash

unzip stocks.zip

In [ ]:
# creating a new spark session with pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
from pyspark.sql.functions import udf, lit, when, date_sub
from pyspark.sql.types import ArrayType, IntegerType, StructType, StructField, StringType
from pyspark.sql.types import BooleanType, DateType, DoubleType
import json
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import Row
from datetime import datetime

# conf = SparkConf().setAppName('Colab').setMaster('local')
# sc = SparkContext(conf=spark)
sqlContext = SQLContext(spark)

/content/spark-3.2.1-bin-hadoop2.7/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
#  Reading all the csv files from stocks dir in colab
from  pyspark.sql.functions import input_file_name
df = spark.read.option("header",True).csv('/content/stocks/*').withColumn("filename", input_file_name())

In [ ]:
df.count()

745076

In [ ]:
df.printSchema()

root
 |-- SC_CODE: string (nullable = true)
 |-- SC_NAME: string (nullable = true)
 |-- SC_GROUP: string (nullable = true)
 |-- SC_TYPE: string (nullable = true)
 |-- OPEN: string (nullable = true)
 |-- HIGH: string (nullable = true)
 |-- LOW: string (nullable = true)
 |-- CLOSE: string (nullable = true)
 |-- LAST: string (nullable = true)
 |-- PREVCLOSE: string (nullable = true)
 |-- NO_TRADES: string (nullable = true)
 |-- NO_OF_SHRS: string (nullable = true)
 |-- NET_TURNOV: string (nullable = true)
 |-- TDCLOINDI: string (nullable = true)
 |-- filename: string (nullable = false)



In [ ]:
# Todo
#  Convert filename EQ081021.CSV to date format, where day= 08 , month = 10, year = 2021

In [ ]:
df.createOrReplaceTempView("temp_df_view")

In [ ]:
spark.sql("select * from temp_df_view limit 5").show()

+-------+------------+--------+-------+-------+-------+-------+-------+-------+---------+---------+----------+------------+---------+--------------------+
|SC_CODE|     SC_NAME|SC_GROUP|SC_TYPE|   OPEN|   HIGH|    LOW|  CLOSE|   LAST|PREVCLOSE|NO_TRADES|NO_OF_SHRS|  NET_TURNOV|TDCLOINDI|            filename|
+-------+------------+--------+-------+-------+-------+-------+-------+-------+---------+---------+----------+------------+---------+--------------------+
| 500002|ABB LTD.    |      A |      Q|1860.25|1892.00|1846.00|1873.20|1864.00|  1854.95|      768|      4120|  7666507.00|     null|file:/content/sto...|
| 500003|AEGIS LOGIS |      A |      Q| 230.25| 233.35| 228.00| 229.35| 229.35|   228.95|     2475|     54056| 12441733.00|     null|file:/content/sto...|
| 500008|AMAR RAJA BA|      A |      Q| 774.50| 774.85| 762.00| 766.15| 766.15|   765.50|     2642|     63238| 48593068.00|     null|file:/content/sto...|
| 500009|A.SARABHAI  |      X |      Q|  30.75|  31.00|  30.55|  30.65

In [ ]:
df.describe().show()

+-------+------------------+------------+--------+-------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+--------------------+---------+--------------------+
|summary|           SC_CODE|     SC_NAME|SC_GROUP|SC_TYPE|             OPEN|              HIGH|               LOW|             CLOSE|              LAST|         PREVCLOSE|        NO_TRADES|        NO_OF_SHRS|          NET_TURNOV|TDCLOINDI|            filename|
+-------+------------------+------------+--------+-------+-----------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+--------------------+---------+--------------------+
|  count|            744838|      744838|  744838| 744838|           744838|            744838|            744838|            744838|            744838|            744838|           744838|            744838|         

In [ ]:
# Todo 
# Create a table in local and connect it with pyspark
# Load data from tempview to table